## Notification Service

In [ ]:
%%writefile configs/notification_service.conf

rabbitmq: {
    ip = "localhost"
    port = 5672
    username = "log6953fe"
    password = "log6953fe"
    vhost = /
    exchange_name = "log6953fe_AMQP"
    exchange_type = "topic"
}
notification: {
    host = "smtp.gmail.com" # INSERT SMTP HOST HERE
    sender = "" # INSERT SMTP CLIENT ID/EMAIL HERE
    password = "" # INSERT SMTP CLIENT ID/EMAIL PASSWORD HERE
    receiver = "" # INSERT RECEIVER EMAIL HERE
}


In [ ]:
%%writefile notification_service.py

import sys
import os

current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'services', 'Current directory is not services'

parent_dir = os.path.dirname(current_dir)

software_dir = os.path.join(parent_dir, 'software')

assert os.path.exists(software_dir), 'software folder not found in the repository root'

sys.path.append(software_dir)

import traceback
import json
import logging
import smtplib
from enum import Enum
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime, timezone
from config.config import load_config
from communication.protocol import ROUTING_KEY_STM_NOTIFICATION
from communication.rpc_server import RPCServer


class VehicleStopStatus(Enum):
    INCOMING_AT	= 1
    STOPPED_AT	= 2
    IN_TRANSIT_TO = 3

    def __str__(self):
        return f"{self.name}"

    @classmethod
    def parse(cls, value: int) -> "VehicleStopStatus":
        try:
            return cls(value)
        except ValueError:
            raise ValueError(f"Invalid VehicleStopStatus value: {value}")


class OccupancyStatus(Enum):
    EMPTY = 1
    MANY_SEATS_AVAILABLE = 2
    FEW_SEATS_AVAILABLE = 3
    STANDING_ROOM_ONLY = 4
    CRUSHED_STANDING_ROOM_ONLY = 5
    FULL = 6
    NOT_ACCEPTING_PASSENGERS = 7

    def __str__(self):
        return f"{self.name}"

    @classmethod
    def parse(cls, value: int) -> "OccupancyStatus":
        try:
            return cls(value)
        except ValueError:
            raise ValueError(f"Invalid OccupancyStatus value: {value}")


logging.config.fileConfig("configs/logging.conf")

class NotificationService(RPCServer):

    def __init__(self, _host:str=None, _sender:str=None, _password:str=None, _rabbitmq_config:dict=None):
        configs = load_config("configs/notification_service.conf")
        self._logger = logging.getLogger("NotificationService")

        if _rabbitmq_config is None:
            self._logger.debug("RabbitMQ config value is empty, reverting to default configs.")
            _rabbitmq_config = configs['rabbitmq']

        self.server = None
        self.host = _host if _host is not None else str(configs['notification']['host'])
        self.sender = _sender if _sender is not None else str(configs['notification']['sender'])
        self.password = _password if _password is not None else str(configs['notification']['password'])
        self.receiver = "test@mail.com" if 'receiver' not in configs['notification'] else str(configs['notification']['receiver'])

        self._logger.debug(f"Host: {self.host}, From: {self.sender}, To: {self.receiver}")

        super().__init__(**_rabbitmq_config)


    def setup(self):
        # Subscribe to any message coming from the STM Telemetry Validation.
        super(NotificationService, self).setup(routing_key=ROUTING_KEY_STM_NOTIFICATION, queue_name=ROUTING_KEY_STM_NOTIFICATION, on_message_callback=self.send_alert)
        self._logger.info("NotificationService setup complete.")


    @staticmethod
    def _build_table_row(event:dict) -> str:
        utc_time = datetime.fromtimestamp(event['vehicle.timestamp'], timezone.utc)
        local_time = utc_time.astimezone()
        local_time = local_time.strftime("%Y-%m-%d %H:%M:%S")

        return f"""
            <tr>
                <td align="center" style="padding: 5px;">{event['vehicle.trip.trip_id']}</td>
                <td align="center" style="padding: 5px;">{event['vehicle.trip.route_id']}</td>
                <td align="center" style="padding: 5px;">{local_time}</td>
                <td align="center" style="padding: 5px;">{event['vehicle.current_bus_stop']}</td>
                <td align="center" style="padding: 5px;">{VehicleStopStatus.parse(event['vehicle.current_status'])}</td>
                <td align="center" style="padding: 5px;">{OccupancyStatus.parse(event['vehicle.occupancy_status'])}</td>
            </tr>
        """


    def _build_message_body(self, events:list) -> str:
        bus_lines = []
        rows = ""
        for e in events:
            rows += self._build_table_row(e)
            bus_lines.append(int(e['vehicle.trip.route_id']))
        bus_lines = set(bus_lines)

        return f"""
            <body style="font-family: 'Poppins', Arial, sans-serif">
                <table width="100%" border="0" cellspacing="0" cellpadding="0">
                    <tr>
                        <td align="center" style="padding: 20px;">
                            <table class="content" width="600" border="0" cellspacing="0" cellpadding="0" style="border-collapse: collapse; border: 1px solid #cccccc;">
                                <!-- Header -->
                                <tr>
                                    <td class="header" style="background-color: #345C72; padding: 40px; text-align: center; color: white; font-size: 24px;">
                                        Société de Transport de Montréal Bus Fleet Digital Twin
                                    </td>
                                </tr>

                                <!-- Body -->
                                <tr>
                                    <td class="body" style="padding: 20px; text-align: left; font-size: 16px; line-height: 1.6;">
                                        Good day Operations Team, <br><br>
                                        This email outlines the need for immediate corrective actions regarding telemetry data for the following bus lines:
                                        <br>
                                        <strong> {", ".join(str(x) for x in bus_lines)}</strong>.
                                        <br><br>
                                        Here is a summary of the issues that required <strong>immediate</strong> actions: <br><br>

                                        <table width="100%" border="1" cellspacing="1" cellpadding="1">
                                            <tr>
                                                <th>Trip-Id</th>
                                                <th>Line</th>
                                                <th>Timestamp</th>
                                                <th>Bus-Stop</th>
                                                <th>Status</th>
                                                <th>Occupancy</th>
                                            </tr>
                                            {rows}
                                        </table>
                                    </td>
                                </tr>

                                <tr>
                                    <td class="body" style="padding-left: 20px; text-align: left; font-size: 16px; line-height: 1.6;">
                                        Please prioritize taking actions to remediate issue(s), if need be confirm the readings of the bus driver. <br><br>
                                        Regards <br><br>
                                    </td>
                                </tr>

                                <!-- Footer -->
                                <tr>
                                    <td class="footer" style="background-color: #333333; padding: 40px; text-align: center; color: white; font-size: 14px;">
                                        Copyright &copy; 2025 | LOG6953FE - Digital Twin Engineering
                                    </td>
                                </tr>
                            </table>
                        </td>
                    </tr>
                </table>
                </body>
        """


    def _build_messages(self, events:list) -> MIMEMultipart:
        msg = MIMEMultipart()
        msg["Subject"] = "[Action Required] for Société de Transport de Montréal Bus Fleet Digital Twin"
        msg["From"] = self.sender
        msg["To"] = self.receiver
        msg["Bcc"] = self.receiver
        msg.attach(MIMEText(self._build_message_body(events), "html"))
        return msg


    def _connect_and_send(self, message:MIMEMultipart):

        try :
            self.server = smtplib.SMTP(self.host, 587)
            self.server.starttls()
            self.server.login(self.sender, self.password)
            self.server.sendmail(self.sender, message['To'], message.as_string())

            self._logger.info(f"Email successfully sent to {message['To']}")

        except Exception as __e:
            traceback.print_tb(__e.__traceback__)
            self._logger.error(f"Error connecting to SMTP server: {str(__e)}")

        finally:
            if self.server is not None:
                self.server.quit()


    def send_alert(self, ch, mthd, prop, json_payload:str):
        self._logger.debug(f"Received JSON payload: {json_payload}")

        try:
            payload_dict = json.loads(json_payload)

            if 'data' not in payload_dict:
                self._logger.error("Service received empty payload to be processed")
                return

            self._connect_and_send(self._build_messages(payload_dict['data']))

        except Exception as __e:
            traceback.print_tb(__e.__traceback__)
            self._logger.error(f"Error attempting to send email: {str(__e)}")


if __name__ == '__main__':

    service = NotificationService()
    service.setup()

    while True:
        try:
            service.start_serving()
        except KeyboardInterrupt:
            exit(0)

        except Exception as e:
            print(f"The following exception occurred: {e}")
            traceback.print_tb(e.__traceback__)
            exit(0)


2025-04-12 11:04:59.133 INFO RPCServer : NotificationService setup complete.
2025-04-12 11:05:10.756 INFO RPCServer : Email successfully sent to carlos.pambo@hotmail.com


In [ ]:
import subprocess
import time
import sys

SERVICE_LOG = "Services.log"

service_proc = subprocess.Popen([sys.executable, "notification_service.py"])

time.sleep(6)

print(f"NotificationService = {service_proc.pid}")

assert service_proc.poll() is None, "NotificationService process has crashed"

In [ ]:
service_proc.terminate()
service_proc.wait()

assert service_proc.returncode is not None, "Process has not exited"

In [ ]:
import os

if not os.path.exists(SERVICE_LOG):
    _f = open(SERVICE_LOG, "w")

with open(SERVICE_LOG, "r") as __f:
    print(__f.read())
